# Step 1:
### - For any new serotypes, add the capsid nucleotide sequence(s) to the serotype_capsids.fa file.
### - Make sure the parameters.yml file is up to date (i.e., this is the same version used in the serotype detection script in openbio). All of the subsequences used to find serotypes in use should be represented. The new serotypes should not be in the parameters.yml file yet.



In [1]:
import yaml
from yaml.loader import SafeLoader
from Bio import SeqIO
from Bio.Seq import Seq


PARAM_FILE_NAME = 'parameters.yml'

def get_kmers(sequence, k):
    ''' Get set of unique kmers of size k per sequence '''
    kmers = set(sequence[pos:pos+k] for pos in range(len(sequence)) if len(sequence) + 1 - pos > k)

    return kmers



## Step 2. Confirm the existing subsequences from parameters.yml are still unique in light of new capsid sequences being added to serotype_capsids.fa

### The output of this cell will identify any sequences that are no longer unique. If any are identified those subsequences should be removed from the parameters.yml file.

In [4]:
# get the existing capsid subsequences from the parameters file
with open(PARAM_FILE_NAME, 'r') as yaml_file:
    capsid_subsequences = yaml.load(yaml_file, Loader=SafeLoader)['serotypes']

# find if existing sequences in capsid_subsequences can be found in any of the full sequences in serotype_capsids.fa (i.e., find if they are still unique).
with open("serotype_capsids.fa") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        record.seq = record.seq.upper()
        for serotype, subsequences in capsid_subsequences['signatures'].items():
            if serotype != record.id: # do not want to compare to itself
                for subsequence in subsequences:
                    subsequence = subsequence.upper()

                    # compare sequences in forward direction, reverse, and reverse complement
                    rev_compl = str(Seq(subsequence).reverse_complement())
                    subseq_loc_rev_compl = record.seq.find(rev_compl)
                    
                    subseq_loc_rev = record.seq.find(subsequence[::-1])
                    
                    subseq_loc = record.seq.find(subsequence)

                    # if one of the subsequences were found (the result is not -1), print out results 
                    if subseq_loc != -1:
                        print(f'The subsequence {subsequence} from serotype {serotype} was found in the capsid sequence for serotype {record.id}.')
                    if subseq_loc_rev != -1:
                        print(f'The reverse of subsequence {subsequence} from serotype {serotype} was found in the capsid sequence for serotype {record.id}.')
                    if subseq_loc_rev_compl != -1:
                        print(f'The reverse complement of subsequence {subsequence} from serotype {serotype} was found in the capsid sequence for serotype {record.id}.')


The subsequence CTTTGGCGGTGCCTTTTAAGGCACAGGCGCAGA from serotype PHPeB was found in the capsid sequence for serotype Cap-B10.
The subsequence CTTTGGCGGTGCCTTTTAAGGCACAGGCGCAGA from serotype PHPeB was found in the capsid sequence for serotype Cap-B22.


## Step 3. Identify new unique subsquences for the new serotypes.

### The below code will identify a unique subsequence for each serotype. First, define the length of the unique subsequences you want produced using the kmer_length variable (default 25 bp). Then, select the number of unique subsequences to output per serotype. Identify the new serotypes in the below list and add the new unique subsequences to parameters.yml in this directory and in the parameters.yml file in openbio.

### In addition, if any sequences were identified in Step 2 that are no longer unique, the new subsequence for that serotype below should be used to replace the existing subsequence. All others subsequences for existing serotypes that are still unique can be left as is (no need to replace with the below new sequence).

In [5]:
kmer_length = 25
max_num_subsequences_to_return = 5

In [6]:
# get the set of kmers for each capsid serotype and store in serotype_kmers
serotype_kmers = {}
with open("serotype_capsids.fa") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        record.seq = str(record.seq.upper())
        serotype_kmers[record.id] = get_kmers(record.seq, kmer_length)

# for each serotype (key in serotype_kmers), find a single subsequence in the values that is not found in the values of any other key in the dictionary
for serotype1,kmer_set1 in serotype_kmers.items():
    combinedkmers = []
    for serotype2,kmer_set2 in serotype_kmers.items():
        if serotype1 != serotype2:
            combinedkmers.extend(kmer_set2) # combine all other values into one list
    
    # find unique sequences for each serotype
    count = 0
    for subseq in kmer_set1:
        # check that the subseq is uniqe in forward direction, reverse, and reverse complement
        if subseq not in combinedkmers and subseq[::-1] not in combinedkmers and str(Seq(subseq).reverse_complement()) not in combinedkmers: 
            count += 1
            if count <= max_num_subsequences_to_return:
                print(serotype1, subseq)
            else:
                break 
    

AAV1 GGAAATTAAAGCCACTAACCCTGTG
AAV1 TACTGCCTGGAATATTTCCCTTCTC
AAV1 ACACCTTTGAGGAAGTGCCTTTCCA
AAV1 AACGGGGGCCAGCAACGACAACCAC
AAV1 TACCTGGCATGGTGTGGCAAGATAG
AAV2 CCTCCTCCACAGATTCTCATCAAGA
AAV2 CTCCTCCACAGATTCTCATCAAGAA
AAV2 TACCTCACCCTGAACAACGGGAGTC
AAV2 ACCCTGAACAACGGGAGTCAGGCAG
AAV2 ACCTCACCCTGAACAACGGGAGTCA
AAV5 GGTTCCCGGGGCCCATGGGCCGAAC
AAV5 AGAGACGGGGGCGCACTTTCACCCC
AAV5 TGCTGCCTGGTTATAACTATCTCGG
AAV5 CGCCTGTGCCCGGAAATATCACCAG
AAV5 GTGTTTACGGACGACGACTACCAGC
AAV8 AAATTCATTGGCTAATCCTGGCATC
AAV8 CCAAATACCATCTGAATGGAAGAAA
AAV8 ACTGCTGGGACCAAATACCATCTGA
AAV8 AAACAAAATGCTGCCAGAGACAATG
AAV8 CCACAGCCAGAGCTTGGACCGGCTG
AAV9 AAACAAGGAACTGGAAGAGACAACG
AAV9 AACTGGAAGAGACAACGTGGATGCG
AAV9 TGAATCCTGGACCTGCTATGGCCAG
AAV9 ACTTCAAGCTCTTCAACATTCAGGT
AAV9 ACAAGGAACTGGAAGAGACAACGTG
AAVrg TCGGACTTAAACACCCTCCTCCCCA
AAVrg CCAGAGAGGCAACCTAGCAGACCAA
AAVrg ACCTCCAGAGAGGCAACCTAGCAGA
AAVrg AACAAGTCTATTAATGTGGACTTTA
AAVrg TACACAAAAACTGCTAGGCAAGCAG
PHPs GGACGTCTTTGGCACAGGCGCAGAC
PHPs TGGCACAGGCGCAGACCGGTTGGGT
PHP